# Hydration free energy

In this notebook you'll learn how to use BioSimSpace to set up a set of _alchemical_ free energy simulations to compute the relative free energy of hydration ethane and methanol. Alchemical free energy calculations employ unphysical ("alchemical") intermediates to estimate the free energies of various physical processes. Here, we will exploit a perturbation pathway between ethane and methanol which will allow us to estimate the relative free energy difference in solution, $\Delta G_{\mathrm{sol}}$, and vacuum, $\Delta G_{\mathrm{vac}}$. The free energy of hydration, $\Delta\Delta G_{\mathrm{hyd}}$, can then be computed from the difference between the free energy of the solvation and vacuum simulation legs, i.e.

$\Delta\Delta G_{\mathrm{hyd}} = \Delta G_{\mathrm{sol}} - \Delta G_{\mathrm{vac}}$

To get started, let's import BioSimSpace and load our ethane and methanol molecules. For convenience, these molecules have already been parameterised with the Generalized Amber Force-Field (GAFF).

In [ ]:
import BioSimSpace as BSS

ethane = BSS.IO.readMolecules(BSS.IO.glob("molecules/ethane.*"))[0]
methanol = BSS.IO.readMolecules(BSS.IO.glob("molecules/methanol.*"))[0]

In order to create a perturbation between the two molecules we need to define a _mapping_. This mapping defines the maximum common substructure (MCS) between the molecules, mapping indices from one to the matching indices in the other. The `BioSimSpace.Align` package provides tools for calculating the mappings between molecules and for aligning and merging molecules based on a mapping. The user can request a multiple mappings, which can be scored and aligned in various ways. Here we'll simply use the default options and compute the highest scoring mapping between the molecules:

In [ ]:
mapping = BSS.Align.matchAtoms(ethane, methanol)

The mapping is just a dictionary mapping atoms in the ethane molecule to those in the methanol. To see what atoms have been mapped we can print the corresponding atoms from each molecule:

In [ ]:
for idx0, idx1 in mapping.items():
    print(ethane.getAtoms()[idx0], "<-->", methanol.getAtoms()[idx1])

Next we need to _align_ the ethane molecule to the methane based on the mapping. The `BioSimSpace.Align.rmsdAlign` function will align them based on a root mean square displacement (RMSD) scoring function, whereas `BioSimSpace.ALign.flexAlign` will flexibly align ethane to methanol using the [fkcombu](http://strcomp.protein.osaka-u.ac.jp/kcombu/doc/README_fkcombu.html) program. For simplicity we'll use the RMSD alignment:

In [ ]:
ethane = BSS.Align.rmsdAlign(ethane, methanol, mapping)

(Note that if we were to omit the `mapping` argument above, then the `rmsdAlign` function would compute the best mapping for us automatically. This means that we can choose to compute the mapping then align separately, or do the whole thing in one step.)

Having succesfully aligned the ethane molecule, we now need to _merge_ the two molecules together to create a _merged_ (or _perturbable_ molecule). This contains all of the properties, e.g. bonds, angles, dihedrals, needed to define the two molecules at either end state of the pertubation, $\lambda=0$ and $\lambda=1$. The end states represent the two molecules (ethane in one state, methanol in the other) plus _dummy_ (or _virtual_) atoms for any atoms from the other molecule that aren't part of the MCS.

In [ ]:
merged = BSS.Align.merge(ethane, methanol, mapping)

To examine the merged molecule we can extract the molecule corresponding to each end state, then look at the elements:

In [ ]:
# Extract the two end states using an internal function.
molecule0 = merged._toRegularMolecule()
molecule1 = merged._toRegularMolecule(is_lambda1=True)

for atom0, atom1 in zip(molecule0.getAtoms(), molecule1.getAtoms()):
    print(atom0._sire_object.property("element"), atom1._sire_object.property("element"))

(Note that the $\lambda=0$ state is just the ethane molecule, whereas the $\lambda=1$ state is the methanol plus two dummy atoms for the two ethane atoms that are not part of the MCS mapping.)

We can also visualise the two end states:

In [ ]:
view = BSS.Notebook.View(molecule0.toSystem())
view.system()

In [ ]:
view = BSS.Notebook.View(molecule1.toSystem())
view.system()

Next we need to solvate our merged molecule in a box of water. Here we'll use the TIP3P water model and a cubic box with a base length of 5 nanometers:

In [ ]:
solvated = BSS.Solvent.tip3p(molecule=merged, box=3*[5*BSS.Units.Length.nanometer])

In order to set up our alchemical simulation we need to a _protocol_ that defines our free energy perturbation. Here we will use the default protocol, which uses 11 $\lambda\$ windows between a value of 0 (the pure ethane end state) and 1 (the pure methanol end state). Choosing a sensible number of windows is extremely important for achieving a stable perturbation and good sampling. If the transformation between windows is too abrupt then the simulation will likely crash. If overlap is poor, then we'll be unable to obtain reliable estimates of the free energy gradient.

In [ ]:
protocol = BSS.Protocol.FreeEnergy()

Finally we can create an solvation free energy object, which will automatically configure everything that is needed to run our simulation. First we will use our built in, GPU optimised, molecular dynamics engine called SOMD. (This is a wrapper around the exellent [OpenMM](http://openmm.org) package and is the default engine if no other packages are present.)

In [ ]:
freenrg_somd = BSS.FreeEnergy.Solvation(solvated, protocol, engine="SOMD", work_dir="freenrg_somd")

When complete, BioSimSpace will have set up a folder hierarchy containing everything that is needed to run the hydration free energy calculation using SOMD. Let's examine the `work_dir` specified above:

In [ ]:
!ls freenrg_somd

Inside the top-level directory are two sub-directories called `free` and `vacuum`. These correspond the the _solvated_ and _vacuum_ legs of the simulation. Let's further examine the `free` directory to see what's inside:

In [ ]:
!ls freenrg_somd/free

Inside this are 11 further sub-directories, one for each of the $\lambda$ windows of the leg. Within each of these directories are all of the files needed to run an individual simulation, e.g.:

In [ ]:
!ls freenrg_somd/free/lambda_0.0000

The `BioSimSpace.FreeEnergy.Solvation` object can also automatically run all of the simulations for you and analyse the output that is generated. However, since these simulations will take a long time we won't run them here.

By specifying a different molecular dynamics engine, we can use the exact same code to set up an identical set of simulations with GROMACS:

In [ ]:
freenrg_gromacs = BSS.FreeEnergy.Solvation(solvated, protocol, engine="GROMACS", work_dir="freenrg_gromacs")

Let's examine the directory for the $\lambda=0$ window of the free leg:

In [ ]:
!ls freenrg_gromacs/free/lambda_0.0000

There you go! This notebook has shown you how BioSimSpace can be used to easily set up everything that is needed for complex alchemical free energy simulations.